In [1]:
import datasets
import numpy
from datasets import load_dataset

dataset : datasets.dataset_dict.DatasetDict = load_dataset("imdb")

Reusing dataset imdb (/home/atomesz/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
x_train : list = dataset["train"]["text"]
y_train : list = dataset["train"]["label"]

x_test : list = dataset["test"]["text"]
y_test : list = dataset["test"]["label"]

In [3]:
from multiprocessing import Pool
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

import functools


def get_gnb_score(preprocess_func, binary:bool = False, remove_stopwords: bool = False):
    """
    Get the score of the Gaussian Naive Bayes
    
    Parameters
    ----------
    preprocess_func : function
        A function from preprocessing: 'basic', 'stemming' or 'lemming'
    
    binary : bool
        A boolean to choose if CountVectorizer count as binary or not
        
    remove_stopwords : bool
        A boolean to choose if the preprocessing function also remove stopwords

    Returns
    -------
    None
        But it print the results of the classification_report (precision, recall, f1-score..) per classes
    """
    # We apply our preprocessing function to the training set
    with Pool() as p:
        preprocessed_x_train : list = p.map(functools.partial(preprocess_func, remove_stopwords=remove_stopwords), x_train)
        
    vectorizer : CountVectorizer = CountVectorizer(max_features=1000, binary=binary)
    X : numpy.ndarray = vectorizer.fit_transform(preprocessed_x_train).toarray()
    
    gnb : GaussianNB = GaussianNB()
    gnb.fit(X, y_train)

    # We apply our preprocessing function to the testing set
    with Pool() as p:
        preprocessed_x_test : list = p.map(functools.partial(preprocess_func, remove_stopwords=remove_stopwords), x_test)
        
    X_test : numpy.ndarray = vectorizer.transform(preprocessed_x_test).toarray()
    
    y_pred : numpy.ndarray = gnb.predict(X_test)
    
    print(classification_report(y_test, y_pred))

In [4]:
from preprocessing import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/atomesz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/atomesz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Without preprocessing

In [5]:
get_gnb_score(basic)

              precision    recall  f1-score   support

           0       0.78      0.84      0.81     12500
           1       0.83      0.77      0.80     12500

    accuracy                           0.80     25000
   macro avg       0.80      0.80      0.80     25000
weighted avg       0.80      0.80      0.80     25000



# Stopwords removal

In [6]:
get_gnb_score(basic, remove_stopwords=True)

              precision    recall  f1-score   support

           0       0.76      0.84      0.80     12500
           1       0.82      0.74      0.78     12500

    accuracy                           0.79     25000
   macro avg       0.79      0.79      0.79     25000
weighted avg       0.79      0.79      0.79     25000



# Stemming

In [7]:
get_gnb_score(stemming, remove_stopwords=False)

              precision    recall  f1-score   support

           0       0.72      0.85      0.78     12500
           1       0.82      0.67      0.74     12500

    accuracy                           0.76     25000
   macro avg       0.77      0.76      0.76     25000
weighted avg       0.77      0.76      0.76     25000



# Stopwords removal + Stemming

In [8]:
get_gnb_score(stemming, remove_stopwords=True)

              precision    recall  f1-score   support

           0       0.73      0.84      0.78     12500
           1       0.81      0.68      0.74     12500

    accuracy                           0.76     25000
   macro avg       0.77      0.76      0.76     25000
weighted avg       0.77      0.76      0.76     25000



# Lemming

In [9]:
get_gnb_score(lemming, remove_stopwords=False)

              precision    recall  f1-score   support

           0       0.74      0.84      0.79     12500
           1       0.82      0.70      0.76     12500

    accuracy                           0.77     25000
   macro avg       0.78      0.77      0.77     25000
weighted avg       0.78      0.77      0.77     25000



# Stopwords removal + Lemming

In [10]:
get_gnb_score(lemming, remove_stopwords=True)

              precision    recall  f1-score   support

           0       0.72      0.85      0.78     12500
           1       0.81      0.68      0.74     12500

    accuracy                           0.76     25000
   macro avg       0.77      0.76      0.76     25000
weighted avg       0.77      0.76      0.76     25000



# This time with binary version

In [11]:
get_gnb_score(basic, binary=True)

              precision    recall  f1-score   support

           0       0.82      0.82      0.82     12500
           1       0.82      0.82      0.82     12500

    accuracy                           0.82     25000
   macro avg       0.82      0.82      0.82     25000
weighted avg       0.82      0.82      0.82     25000



In [12]:
get_gnb_score(basic, binary=True, remove_stopwords=True)

              precision    recall  f1-score   support

           0       0.81      0.82      0.82     12500
           1       0.82      0.81      0.81     12500

    accuracy                           0.81     25000
   macro avg       0.81      0.81      0.81     25000
weighted avg       0.81      0.81      0.81     25000



In [13]:
get_gnb_score(stemming, binary=True, remove_stopwords=False)

              precision    recall  f1-score   support

           0       0.80      0.82      0.81     12500
           1       0.82      0.79      0.80     12500

    accuracy                           0.81     25000
   macro avg       0.81      0.81      0.81     25000
weighted avg       0.81      0.81      0.81     25000



In [14]:
get_gnb_score(stemming, binary=True, remove_stopwords=True)

              precision    recall  f1-score   support

           0       0.80      0.82      0.81     12500
           1       0.82      0.80      0.81     12500

    accuracy                           0.81     25000
   macro avg       0.81      0.81      0.81     25000
weighted avg       0.81      0.81      0.81     25000



In [15]:
get_gnb_score(lemming, binary=True, remove_stopwords=False)

              precision    recall  f1-score   support

           0       0.80      0.82      0.81     12500
           1       0.81      0.79      0.80     12500

    accuracy                           0.80     25000
   macro avg       0.80      0.80      0.80     25000
weighted avg       0.80      0.80      0.80     25000



In [16]:
get_gnb_score(lemming, binary=True, remove_stopwords=True)

              precision    recall  f1-score   support

           0       0.79      0.82      0.81     12500
           1       0.81      0.79      0.80     12500

    accuracy                           0.80     25000
   macro avg       0.80      0.80      0.80     25000
weighted avg       0.80      0.80      0.80     25000



# Results analysis

The best results are obtained without preprocessing and counting if words are present or not

In [24]:
preprocess_func = basic
binary: bool = True
remove_stopwords: bool = False

# We apply our preprocessing function to the training set
with Pool() as p:
    preprocessed_x_train : list = p.map(functools.partial(preprocess_func, remove_stopwords=remove_stopwords), x_train)

vectorizer : CountVectorizer = CountVectorizer(max_features=1000, binary=binary)
X : numpy.ndarray = vectorizer.fit_transform(preprocessed_x_train).toarray()

gnb : GaussianNB = GaussianNB()
gnb.fit(X, y_train)

# We apply our preprocessing function to the testing set
with Pool() as p:
    preprocessed_x_test : list = p.map(functools.partial(preprocess_func, remove_stopwords=remove_stopwords), x_test)

X_test : numpy.ndarray = vectorizer.transform(preprocessed_x_test).toarray()

y_pred : numpy.ndarray = gnb.predict(X_test)

In [26]:
from analyze import find_wrong_class

few_wrong = find_wrong_class(x_test, y_pred, y_test)

In [29]:
few_wrong[:2]

[("I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first half of the movie, and were moved to tears during the second half. While exiting the theater I not only saw many women in tears, but many full grown men as well, trying desperately not to let anyone see them crying. This movie was great, and I suggest that you go see it before you judge.",
  0),
 ("I first saw this movie on IFC. Which is a great network by the way to see underground films. I watched this movie and was thinking it was going to be pure drama and a story line that doesn't 

If we take those two samples that were mispredicted, we can see that they uses negative words as well so their feelings are more implicit